In [ ]:
import numpy as np
import torch
from datetime import datetime
import sys, os, math
import torch.nn as nn
import torch.cuda as cd
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import *
from torch.autograd import Variable
import re
import torch.nn.functional as F
import Levenshtein
import random


# 1. Data preprocessing

I have converted all the labels so that the model can fall to character level. The files are processed by another computer which has more memory than my AWS instance.

In [ ]:
charSet = ['~', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', \
               'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-', "'", '.', '_', '+', ' ','%','&']# ~ is <pad>, % is <sos>, & is <eos>
charSetWithoutBlank = charSet[1:]
charSetDict = {charSet[i]:i for i in range(len(charSet))}

In [ ]:
trainX = torch.load("train_x.pt")
trainY = torch.load("train_y.pt")
devX = torch.load("dev_x.pt")
devY = torch.load("dev_y.pt")
trainX_lens = torch.load("train_x_lens.pt")
trainY_lens = torch.load("train_y_lens.pt")
devX_lens = torch.load("dev_x_lens.pt")
devY_lens = torch.load("dev_y_lens.pt")

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#DEVICE = "cpu"

## The Vocal dataset

a subclass that returns the selected data's actual lengths after padding. This is essential for ctcdecode and the loss.

In [ ]:
class VocalDataset(Dataset):

    def __init__(self, x, y, xlen, ylen):
        super().__init__()
        self._x = x
        self._y = y
        self.xlens = xlen
        self.ylens = ylen if y is not None else [0 for i in len(x)]
        print(len(x), len(y) if y is not None else -1)

    def __getitem__(self, index):
        xt = self._x[index]
        yt = self._y[index] if self._y is not None else [0]

        return xt, yt, self.xlens[index], self.ylens[index]

    def __len__(self):
        return len(self._x)

def collate_fn(batch_data):
    # batch_data [x, xlen, y, ylen]
    # x [batch_size, -] -> [batch_size, max_len, 40]
    # xlen [batch_size]
    # y [batch_size, -] -> [batch_size, max_len]
    # ylen [batch_size]

    x_ret = [b[0] for b in batch_data]
    x_len = torch.LongTensor([b[2] for b in batch_data])
    y_ret = [b[1] for b in batch_data]
    y_len = torch.LongTensor([b[3] for b in batch_data])

    x = pad_sequence(x_ret, batch_first=True)
    y = pad_sequence(y_ret, batch_first=True).to(torch.long)

    return x, y, x_len, y_len


In [ ]:
batch_size = 64
#batch_size = 1

trainP = VocalDataset(trainX, trainY, trainX_lens, trainY_lens)
trainT = DataLoader(trainP, batch_size=batch_size, shuffle=True, num_workers=16, collate_fn=collate_fn) #pin_memory=True

devP = VocalDataset(devX, devY, devX_lens, devY_lens)
devT = DataLoader(devP, batch_size=batch_size, shuffle=False, num_workers=16, collate_fn=collate_fn)

24724 24724
1106 1106


# 2. Model Formulations

## Locked Dropout

This dropout constrains the part of sequence to be dropped out at the same place of the sequence, instead of happening randomly between LSTM layers which beats the consistency between the outputs.

code from [Salesforce](https://github.com/salesforce/awd-lstm-lm/blob/32fcb42562aeb5c7e6c9dec3f2a3baaaf68a5cb5/locked_dropout.py#L5)

In [ ]:
class LockedDropout(nn.Module):
    def __init__(self):
        super(LockedDropout,self).__init__()

    # ...
    def forward(self, x, dropout):
        if not self.training or dropout == 0:
            return x
        m = x.data.new(x.size(0), 1, x.size(2)).bernoulli_(1 - dropout)
        mask = Variable(m, requires_grad=False) / (1 - dropout)
        mask = mask.expand_as(x)
        return mask * x

In [ ]:
def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape).to(DEVICE)
    return -Variable(torch.log(-torch.log(U + eps) + eps))

def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)

def GumbelSoftmax(logits, temperature):
    """
    input: [*, n_class]
    return: [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)
    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    return (y_hard - y).detach() + y

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention,self).__init__()

    def forward(self, query, key, value, lens):
        '''
        :param query :(batch_size, key_size) Query is the output of LSTMCell from Decoder
        :param key: (batch_size, key_size) Key Projection from Encoder per time step
        :param value: (batch_size, value_size) Value Projection from Encoder per time step
        :param lens: (batch_size) boolean marking the end of the array or not
        :return output: Attended Context (batch_size, value_size)
        :return attention_mask: Attention mask that can be plotted  (batch_size, max_len=1)
        '''
        # key (batch_size, seq_len, key_size)
        # query (batch_size, key_size, 1)
        # value (batch_size, seq_len, value_size)

        batch_size = query.shape[0]
        key_size = key.shape[2]
        value_size = value.shape[2]
        seq_len = key.shape[1]

        # energy = bmm(key, query) (batch_size, seq_len, key_size), (batch_size, content_size, 1) -> (batch_size, seq_len, 1)
        #energy = torch.einsum("ijk,ikl->ijl",key.view(batch_size, -1, key_size),query.view(batch_size, key_size, 1)).view(batch_size, seq_len)
        energy = torch.bmm(key.view(batch_size, -1, key_size),query.view(batch_size, key_size, 1)).view(batch_size, seq_len)

        if lens is not None:
            mask = torch.arange(seq_len).to(DEVICE).view(1,-1) >= lens.view(-1,1) #(batch_size, seq_len)
            energy.masked_fill_(mask, -1145141919)
        attention = F.softmax(energy,dim=1)
        #print(attention.shape, value.shape)
        # context = bmm(attention, value) (batch_size,1,seq_len), (batch_size,seq_len,value_size) -> (batch_size,1,value_size)
        #context = torch.einsum("ijk,ikl->ijl",attention.view(batch_size, 1, -1),value)
        context = torch.bmm(attention.view(batch_size, 1, -1),value)
        return context.view(batch_size, value_size), attention

In [ ]:
class Encoder(nn.Module):
    def __init__(self, char_size=41, input_size=40, hidden_size=256, num_layers=3, dropout=0.0, bidirectional=True, key_size=128, value_size=128):
        super(Encoder,self).__init__()
        self.input_size = input_size
        self.char_size = char_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.key_size = key_size
        self.value_size = value_size
        self.direction = 2 if bidirectional==True else 1
        self.drp = dropout
        self.dropout = nn.Dropout(p=dropout)
        # batch_first implementation is more intuitional
        self.rnn0 = nn.LSTM(input_size,hidden_size,1,dropout=dropout,bidirectional=bidirectional,batch_first=True)
        self.LckDropout = LockedDropout()
        self.pyramid = nn.ModuleList([
            nn.LSTM(2 * hidden_size * self.direction, hidden_size, 1, dropout=dropout, bidirectional=bidirectional,batch_first=True) for _ in range(num_layers)
        ])

        self.key_fc = nn.Linear(self.hidden_size * self.direction, self.key_size)
        self.value_fc = nn.Linear(self.hidden_size * self.direction, self.value_size)


    def shrink(self, x, x_len):
        batch_size = x.shape[0]
        next_seq_len = x.shape[1] // 2
        depth = x.shape[2]
        # [batch_size, seq_len, hidden_size] -> [batch_size, floor(seq_len / 2), hidden_size * 2]
        # slicing before cutting
        x = x[:,:next_seq_len * 2,:].contiguous().view(batch_size, next_seq_len, depth * 2)
        x_len = x_len // 2
        return x, x_len


    def forward(self, x, x_len): # x(pad_x [batch_size, seq_len, 40], pad_len [batch_size])

        batch_size = x.shape[0]
        seq_len = x.shape[1]
        x = pack_padded_sequence(x, x_len, enforce_sorted=False,batch_first=True)
        x, h = self.rnn0(x)
        for i in range(self.num_layers):
            x, x_len = pad_packed_sequence(x,batch_first=True)
            x = self.LckDropout(x, self.drp)
            x, x_len = self.shrink(x, x_len)
            x = pack_padded_sequence(x, x_len, enforce_sorted=False,batch_first=True)
            x, h = self.pyramid[i](x)

        x, x_len = pad_packed_sequence(x,batch_first=True)
        seq_len = x.shape[1]
        x = self.dropout(x)
        x = x.view(batch_size * seq_len,-1) #(batch_size * seq_len // 8, hidden_size)
        keys = self.key_fc(x).view(batch_size, seq_len, -1)
        values = self.value_fc(x).view(batch_size, seq_len, -1)

        return keys, values


In [ ]:
class Decoder(nn.Module):
    def __init__(self, char_size=41, hidden_size=512, value_size=128, key_size=128, attention=False, noise=False):
        super(Decoder,self).__init__()
        self.char_size = char_size
        self.hidden_size = hidden_size
        self.key_size = key_size
        self.value_size = value_size
        self.embedding = nn.Embedding(char_size, hidden_size, padding_idx=0)
        self.lstm1 = nn.LSTMCell(hidden_size + value_size, hidden_size) # auto batch_size first for lstmcell
        self.lstm2 = nn.LSTMCell(hidden_size, key_size)

        self.att = attention
        if self.att: self.attention = Attention()

        self.noise = noise
        self.temp = 1

        self.fc = nn.Linear(key_size + value_size, char_size)

    def forward(self, key, values, text=None, text_lens=None, teacher_frc=0.0):
        # key [batch_size, seq_len, key_size]
        # value [batch_size, seq_len, value_size]

        isTrain = self.training
        batch_size = key.shape[0]

        if isTrain:
            max_len = text.shape[1]
            embeddings = self.embedding(text)
        else:
            max_len = 299

        predictions = []
        hidden_states = [None, None]
        prediction = torch.zeros(batch_size,self.char_size).to(DEVICE)
        prediction[:, charSetDict['%']] = 1. # assume first character is <sos>
        predictions.append(prediction.unsqueeze(1)) #align

        context = torch.zeros(batch_size, self.value_size).to(DEVICE)
        for i in range(max_len - 1):
            teacher_forcing = True if random.random() < teacher_frc else False
            if (isTrain and teacher_forcing):
                char_embed = embeddings[:,i,:] #(batch_size, 1, hidden_size)
            else:
                char_embed = self.embedding(prediction.argmax(dim=-1)) #(batch_size, 1, hidden_size)

            if self.noise:
                char_embed = GumbelSoftMax(char_embed, self.temp)
                #char_embed = F.gumbel_softmax(char_embed,tau=self.tau,hard=True)

            inp = torch.cat([char_embed, context], dim=1) # (batch_size, 1, hidden_size + value_size)
            hidden_states[0] = self.lstm1(inp, hidden_states[0])

            inp_2 = hidden_states[0][0]
            hidden_states[1] = self.lstm2(inp_2, hidden_states[1])

            ### Compute attention from the output of the second LSTM Cell ###
            output = hidden_states[1][0] # (batch_size, 1, key_size)
            if self.att:
                context, attn = self.attention(output, key, values, text_lens)
            else:
                context = values[:,i,:]

            prediction = self.fc(torch.cat([output, context], dim=1))
            predictions.append(prediction.unsqueeze(1))

            if not self.att and i > values.shape[1]: break

        return torch.cat(predictions, dim=1) #(batch_size, max_len, output_size)

In [ ]:
class LAS(nn.Module):
    def __init__(self, input_size=40, char_size=38, hidden_size_e=256, hidden_size_d=512, value_size=128, key_size=128, isAttended=False, noise=False):
        super(LAS,self).__init__()
        self.encoder = Encoder(char_size, input_size, hidden_size_e, 3, 0.1, True, key_size, value_size)
        self.decoder = Decoder(char_size, hidden_size_d, value_size, key_size, isAttended, noise)

    def forward(self, x, x_len, y, y_len,teacher_frc):
        k,v = self.encoder(x, x_len)
        if self.training:
            pred = self.decoder(k,v,y,y_len,teacher_frc)
        else:
            pred = self.decoder(k,v)
        return pred


In [ ]:
num_classes = 47
input_dim = 40
output_dim = len(charSet)
print(output_dim)
hidden_dim_e = 256
hidden_dim_d = 512
value_dim = 128
key_dim = 128

model = LAS(input_dim, output_dim, hidden_dim_e, hidden_dim_d, value_dim, key_dim, isAttended=True, noise=False).to(DEVICE)

#criterion = nn.CrossEntropyLoss(reduction=None)
criterion = nn.CrossEntropyLoss(reduction='sum').to(DEVICE)

35


/home/toni/.local/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [ ]:
def distance(output, label, m, n):
    # Create a table to store results of subproblems

    dp = [[0 for x in range(n + 1)] for x in range(2)]

    # Fill d[][] in bottom up manner
    for i in range(m + 1):
        for j in range(n + 1):

            # If first string is empty, only option is to
            # insert all characters of second string
            if i == 0:
                dp[1][j] = j    # Min. operations = j

            # If second string is empty, only option is to
            # remove all characters of second string
            elif j == 0:
                dp[1][j] = i    # Min. operations = i

            # If last characters are same, ignore last char
            # and recur for remaining string
            elif output[i-1] == label[j-1]:
                dp[1][j] = dp[0][j-1]

            # If last character are different, consider all
            # possibilities and find minimum
            else:
                dp[1][j] = 1 + min(dp[1][j-1],        # Insert
                                   dp[0][j],        # Remove
                                   dp[0][j-1])    # Replace

        dp.pop(0)
        dp.append([0 for x in range(n + 1)])

    return dp[0][n]

In [ ]:
def train(model, optimizer, scheduler, criterion, name, num_epochs, teacher_frc=0.1, pass_first=False):
    total_step = len(trainT)
    #torch.autograd.set_detect_anomaly(True)

    for epoch in range(num_epochs):
        if scheduler is not None:
            print('Epoch:', epoch,'LR:', scheduler.get_lr())

        i = 0
        model.train()
        avg_loss = 0.
        perplexity = 0.
        time_flag=datetime.now()

        if not pass_first:
            for dta in tqdm(trainT):
                #Set the inputs to the device.
                voc, lbl, voc_len, lbl_len = dta
                bs = lbl.shape[0]
                width = lbl.shape[1]
                voc = voc.to(DEVICE)
                lbl = lbl.to(DEVICE)
                voc_len = voc_len.to(DEVICE)
                lbl_len = lbl_len.to(DEVICE)


                # Forward pass
                optimizer.zero_grad()
                outputs = model(voc, voc_len, lbl, lbl_len, teacher_frc)

                output_loss = torch.cat([outputs[i,1:lbl_len[i]] for i in range(bs)],dim=0)
                lbl_loss = torch.cat([lbl[i,1:lbl_len[i]] for i in range(bs)],dim=0)

                loss = criterion(output_loss, lbl_loss)
                loss = loss / lbl_loss.shape[0]
                #mask = torch.arange(width).view(-1,1) < lbl_len.view(1,-1)
                #loss = criterion(outputs.view(-1,output_dim), lbl.view(-1))
                #loss = criterion(output, label, reduce=False) #shape (batch_size, width)
                #mask_output = mask * loss
                #tot_len = (mask == True).nonzero().view(-1).shape[0]
                # use ignore_index should suffice to not use masked output
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
                optimizer.step()
                #avg_loss += math.exp(loss.item() / (lbl_len.sum().item()))
                avg_loss += loss.item()


                if (i+1) % 50 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Per-char Loss: {:.4f}'
                           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                    print("Time Elapsed: ", datetime.now()-time_flag)


                i += 1
                del voc, voc_len
                del lbl, lbl_len

            print('Epoch [{}/{}], Step [{}/{}], Per-char Loss Average: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, avg_loss / total_step))
            print("Time Elapsed: ", datetime.now()-time_flag)

        pass_first=False

        model.eval()
        ed_tot = 0
        tot_dev_size = 0
        cd.empty_cache()
        optimizer.zero_grad()
        train_loss_fin = avg_loss / total_step
        avg_loss = 0.
        batches = 0

        with torch.no_grad():
            for dta in tqdm(devT):
                voc, lbl, voc_len, lbl_len = dta
                bs = lbl.shape[0]
                voc = voc.to(DEVICE)
                lbl = lbl.to(DEVICE)
                voc_len = voc_len.to(DEVICE)
                lbl_len = lbl_len.to(DEVICE)

                mx_lbl_len = lbl.shape[1]

                outputs = model(voc, voc_len, lbl, lbl_len, 0)
                #mask = torch.arange(width).view(-1,1) < lbl_len.view(1,-1)
                output_loss = torch.cat([outputs[i,1:lbl_len[i]] for i in range(bs)],dim=0)
                lbl_loss = torch.cat([lbl[i,1:lbl_len[i]] for i in range(bs)],dim=0)

                loss = criterion(output_loss, lbl_loss)
                avg_loss += loss / lbl_loss.shape[0]

                predict = F.softmax(outputs)
                labels = torch.argmax(predict,dim=-1)
                for i in range(voc.shape[0]):
                    xt = "".join([charSet[k] for k in labels[i][1:lbl_len[i]]])
                    yt = "".join([charSet[k] for k in lbl[i][1:lbl_len[i]]])
                    print("Predicted:",xt,"\nTruth    :",yt)
                    ed = distance(xt,yt,lbl_len[i]-1,lbl_len[i]-1)
                    ed_tot += ed

                tot_dev_size += voc.shape[0]
                batches += 1


            print ('dev avg distance: %lf, avg loss: %lf' % (ed_tot / tot_dev_size , avg_loss / batches))

        cd.empty_cache()
        optimizer.zero_grad()

        #return None

        torch.save(model.state_dict(), name+'-'+str(epoch+1)+'-'+str(datetime.now().strftime("%m%d%H%M"))+'.pkl')

        if scheduler is not None:
            scheduler.step()

        if train_loss_fin < 0.4 and teacher_frc > 0.1: teacher_frc -= 0.16



In [ ]:
torch.autograd.set_detect_anomaly(False)

In [ ]:
train(model, optimizer, None, criterion, "take2",11,0.3,True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Predicted: &~E&F.MALE-PRODUQED-A+LITTER OF&TWW T.YFOU~ YO+NG+IS NOVEMBERXA+D+DECEMBER& 
Truth    : THE FEMALE PRODUCES A LITTER OF TWO TO FOUR YOUNG IN NOVEMBER AND DECEMBER&
Predicted: NUMEJ~US.QORKK&OF ARG.ARE BAKKD.ON THE STORY&VF THE C.-QIFICE V~~~SAAC& 
Truth    : NUMEROUS WORKS OF ART ARE BASED ON THE STORY OF THE SACRIFICE OF ISAAC&
Predicted: THEIR&SOVUMU%N-REQUIRES&DEVELVPMENT&OFFTHE-HUMAN KAPICIZY'FOR SOCIAL INTRRVST& 
Truth    : THEIR SOLUTION REQUIRES DEVELOPMENT OF THE HUMAN CAPACITY FOR SOCIAL INTEREST&
Predicted: YIS&MOSQ SIGNFFICANT_SCHENTIFYC QUBLICATI+NS&WERE.STUDDEZ&OF BVRDZ+A+% ANQMALS& 
Truth    : HIS MOST SIGNIFICANT SCIENTIFIC PUBLICATIONS WERE STUDIES OF BIRDS AND ANIMALS&
Predicted: IN RECENT YEARS&SHE-HAS PPIMARILY APPEAR-D IN TELVVISION FIXMS SUCC JS LITTLE-GLOBIA- 
Truth    : IN RECENT YEARS SHE HAS PRIMARILY APPEARED IN TELEVISION FILMS SUCH AS LITTLE GLORIA&
Predicted: __E_PROCESS BY+WHICH T+E LEN' FOCUVES ON KXXERNAL OBJEXTS&IS KALLEDGBCCOMMUDATION_ 
Trut

Predicted: THERE'I'''NNQDD'JFZOI+N'QQCNOJY'QJ.J.XC_-BBBBGTH+_GUR'UB+..YDVWWW%WFFWWYYQHPPP'P'N'YFH 
Truth    : THERE IS AN OLD FASHIONED ECONOMIST DESCRIBING THE GARDEN OF EDEN WITHOUT THE SERPENT&
Predicted: T_H CIVVVVD+MK'Z.KQ.Z..Z..GVJVVNMENT&&~ 
Truth    : THE SERPENT IN THIS CASE IS GOVERNMENT&
Predicted: IT'WS NOT MMMQICA''PPIVATE CZVZZZZZ'WWV'V'FG'SQGHUA'WOPKPUPPZXXZXYYJ..LYYV. 
Truth    : IT IS NOT AMERICAN PRIVATE CITIZENS WHO HAVE SUCH A LOW PROPENSITY TO SAVE&
Predicted: IT WS T.....&S.'JGVVVNMENB THAT'HZZ''.H.YK'PPPV+XN+DIGTO RUNDJ.PFXIT' THAA'IH'TWWNXGAGJ JGJAGJJJJGKJGJGGGKJ 
Truth    : IT IS THE U. S. GOVERNMENT THAT HAS A HIGH PROPENSITY TO RUN DEFICITS THAT IS TO ENGAGE IN NEGATIVE SAVING&
Predicted: ADDDDD.WSKKN LOGGKR FORMIG+PPPVVAVE'X.XYZZZZWJHWWJA''_TW UNVVXTDI. TH..U....BB.VFUE'A'DGGVVRNMENT' THAT'ARE DOING THE IVVESTING HEREBBY SUPPPRTING THE DOLLAR&BU 
Truth    : AND IT IS NO LONGER FOREIGN PRIVATE CITIZENS WHO WANT TO INVEST IN THE U. S. BUT FOREIGN GOVERNMENTS T

Predicted: DD+'Q--QUOTE ''' J FIRMEB+LVBVBV_IND+VV.STGNGFGORDDREHJ%NNVVVI.DX.MMMWWRWB.QQZZE'Y.RYZZ'TERESTED OO HAHE ' STKKKNIN THE INVESTMENT COMMA DOUBLE-QUOTE SHE SAYS PERIOD&_O 
Truth    : DOUBLE-QUOTE I'M A FIRM BELIEVER IN INVESTING FOR THE FUN OF IT COMMA OR BECAUSE YOU'RE INTERESTED OR HAVE A STAKE IN THE INVESTMENT COMMA DOUBLE-QUOTE SHE SAYS PERIOD&
Predicted: TWOTHAT'V.D.C.MJ%CH..'ZZZ'ZXVESTMV&JNJJZRJ_ZRSHHV''FINAZCING&JWWKLWWUW~KWFIXMS PEEIO 
Truth    : TO THAT END COMMA SHE HAS INVESTED IN PARTNERSHIPS FINANCING HOLLYWOOD FILMS PERIOD&
Predicted: SHHAALSO HAS BAKGHTZMMME SHXZEZOFFFS..LZLL..HPY--BB_ZHD'K+RDZ'-J-FF+--RRP+PYTEDCCOMMA U'DU'PCKLK'JDPAPTMENT'DTOREHHNAGKCOMMA-DOUBQE-QUOTE BECAUSE I VVYYBA'X''''MMUZS'AFFSTFFF THRRR PRRIOD DOUBLE-QUOTE& 
Truth    : SHE ALSO HAS BOUGHT SOME SHARES OF SEATTLE HYPHEN BASED NORDSTROM INCORPORATED COMMA AN UPSCALE DEPARTMENT STORE CHAIN COMMA DOUBLE-QUOTE BECAUSE I BUY LOTS AND LOTS OF STUFF THERE PERIOD DOUBLE-QUOTE&
Predicted: R%CC%+BJ

Predicted: &~&-STAXUE&OF BIBERTY&A.D ELLIS-ISLAND ARE WITHIN THE NEWJJ.RSEY WATHRS&OF KEW YORK B.Y& 
Truth    : THE STATUE OF LIBERTY AND ELLIS ISLAND ARE WITHIN THE NEW JERSEY WATERS OF NEW YORK BAY&
Predicted: &~U-UNIQED.STXTES&UNDERTOOK&WO DEFEND WESTHRN EUROPE&AGAINST SOVVE- JCWAXK& 
Truth    : THE UNITED STATES UNDERTOOK TO DEFEND WESTERN EUROPE AGAINST SOVIET ATTACK&
Predicted: HE NEVVW.OBJAINED A+JQXURI AQADEMPCUPOSITION-OR PERJANENG++MPJY-MENT& 
Truth    : HE NEVER OBTAINED A SECURE ACADEMIC POSITION OR PERMANENT EMPLOYMENT&
Predicted: THEY'ESX-BLISHED&+OYML-C+MMISSIONS&TO&REQOVERYILYEGALLY HELD CHURCH LANDS& 
Truth    : THEY ESTABLISHED ROYAL COMMISSIONS TO RECOVER ILLEGALLY HELD CHURCH LANDS&
Predicted: T~FFFF.T.WMJW.K&Y.-O.DDMJJ..Z..Z..M.XXXXL_XKZZKQKKK'ZZZZZZ.ZXXMMJF-FO-OTWENTY PPJKVZZZZZJ 
Truth    : THE FIFTY ONE YEAR OLD MR. SCIARRA MAKES NO SECRET OF HIS ADMIRATION FOR TONY PROVENZANO&
Predicted: HWWWAS&GR-AK FRR LAB&R& 
Truth    : HE WAS GREAT FOR LABOR&
Predicted: HE&

...

Predicted: TFFFFUNDPQPEVVUQSLYYPPPOSED ONLY KWWWYWWSSWWHTHH SUUGHFAFRWDD''GBMM__+PX+.PPPCCVYBYYYHPPPPZS AI LLATTYY MMMMAWWH'SPKKEMMAN SAID PERIOD&-&PPRIOD&PP-&PE 
Truth    : THE FUND PREVIOUSLY OPPOSED ONLY NEW LOANS TO THE SOUTH AFRICAN GOVERNMENT COMMA SPECIFICALLY THE POLICE AND MILITARY COMMA THE SPOKESMAN SAID PERIOD&
Predicted: WYKNWWWW.CMMM%...FEXER'LGGGVVRNME+T'WZ''''' XXQUALLLLSADZZ 'ZZZZMMMNLLYYGAUUSUPP FXUFFFRZZZZZEXPMPPMITHMMMTRITHPMMFRIMHOFGZXYZHV 
Truth    : RIGHT NOW COMMA THE FEDERAL GOVERNMENT DOESN'T ACTUALLY SET ASIDE MONEY IN A SEPARATE FUND FOR ITS EMPLOYEES' RETIREMENT PERIOD&
Predicted: +UTUU.DER THE KEW FHI~FT'PLLW C+MMMBGUJJJJJJJGJJ.FF'RLL..LYPPXYZZY.CL+'SHUUZZUTPPBUPKHHEYR'ARETTRMMMNNDSVVVZGS INSSTOCK WRBBBNFFFUNDS PERIOD&BOOODDD 
Truth    : BUT UNDER THE NEW THRIFT PLAN COMMA BEGINNING IN JANUARY FEDERAL EMPLOYEES CAN CHOOSE TO PUT THEIR RETIREMENT SAVINGS IN STOCK OR BOND FUNDS PERIOD&
Predicted: SINCK&H+EGGOVVRNGMHMMMM_JJZ'PP+AKFVFF+HH-IMPLYY'''''C++-+BUIZZ

Predicted: U.USUA%LY&HIGG PVVVLS&OF%FADIATION&WE'D DEBECTED IN BANY&UQROPEAX C++N~-IX-& 
Truth    : UNUSUALLY HIGH LEVELS OF RADIATION WERE DETECTED IN MANY EUROPEAN COUNTRIES&
Predicted: BVKH PETROLE-M&-.~XN.TURAL J.S JEPOSITS&ARB QCATTERED THRHUGH Q.STERN O.WO& 
Truth    : BOTH PETROLEUM AND NATURAL GAS DEPOSITS ARE SCATTERED THROUGH EASTERN OHIO&
Predicted: FFRFTHE FIR'T&TIME&~N YEARS&T~+-REPUBLICANS&ALSO CAPTURED BOTZUZOUSES OF CONJRESS& 
Truth    : FOR THE FIRST TIME IN YEARS THE REPUBLICANS ALSO CAPTURED BOTH HOUSES OF CONGRESS&
Predicted: X&H SOUTH CAROL~NA EDUCQTIOL-L RADIO NEWWWKK KAS WOM'NATIZNAL BROWDCASTI+G+BWAYDSK 
Truth    : THE SOUTH CAROLINA EDUCATIONAL RADIO NETWORK HAS WON NATIONAL BROADCASTING AWARDS&
Predicted: A KXKKYR IS ABJH.PPDESGGNEG DY KARRY&YARGG VOLLMES OF OIL OR OTHER QIQQID QAR~OX 
Truth    : A TANKER IS A SHIP DESIGNED TO CARRY LARGE VOLUMES OF OIL OR OTHER LIQUID CARGO&
Predicted: X~E&ENVRMOUM AMOUNTS&OF CARB~B&_.XXIDEXIN THE +TMOZKHER''CAUSZ THIS HIGH P

Predicted: F+RGZGN_PPVVADENINVBSTOR ZS'ANKKQQALSZM'ZPRZZZZV'FOFFFFING''FFCEC''WO POTENTIALPPITFFFFSUCCRRENCY RISKKAND INTEREST'RATE RIS 
Truth    : FOREIGN PRIVATE INVESTORS IN UNCLE SAM'S PRESENT OFFERINGS FACE TWO POTENTIAL PITFALLS CURRENCY RISK AND INTEREST RATE RISK&
Predicted: SZNCZ TH+Y'ARE CJKIINGK'HEIRPPRRFIT INDLWSSIVFVVANHTHC'C+RRENZYXANDDYECKING IN THHHVVLVV FFFTHHHDLLLAR WWLLRREJUCE THEIR AAIEFOF REJUR 
Truth    : SINCE THEY ARE TAKING THEIR PROFIT AND LOSS IN ANOTHER CURRENCY A DECLINE IN THE VALUE OF THE DOLLAR WILL REDUCE THEIR RATE OF RETURN&
Predicted: GFFJ. .. INNEREST___T-' KGLJUU.QHE VALLVVZZLUVFOFFHTEI'RU. S..TREASYRYYHOLDINGSWWI'LGGG WOWNNYVEN FFFTHE DOLLA'''''SABLS&P 
Truth    : IF U. S. INTEREST RATES GO UP THE DOLLAR VALUE OF THEIR U. S. TREASURY HOLDINGS WILL GO DOWN EVEN IF THE DOLLAR IS STABLE&
Predicted: AMMVBBVIOUSLY FF THE DOLLAR G_'S DOWWWAND.U....'DINNJR'RR_D_AIK''''KPPUTHHH WILL 'H''IR'SHHRPZ&&& 
Truth    : AND OBVIOUSLY IF THE DOLLAR GOES DOWN AND U.

Predicted: .YV.OVVR CO++A&F.LLMMEM.RMMS SIIDDBHHTWWWWNTYYFIVVPPERCENT OFFTHE SURVVY BESPONDSSCSWWW'EKNONYHYPHHNHMMKKERS PERIOD& 
Truth    : MOREOVER COMMA PHILIP MORRIS SAID THAT TWENTY FIVE PERCENT OF THE SURVEY RESPONDENTS WERE NON HYPHEN SMOKERS PERIOD&
Predicted: A SPKKKSMMNNFRRKPAPXICCN..INCPPPORATEDSSADD COLNN DOUBL--QUQQQ WHYHVVE FNW L.QITTDPLLANS TTGG. TO SUUKKRRWZZPRPP 
Truth    : A SPOKESMAN FOR PEPSICO INCORPORATED SAID COLON DOUBLE-QUOTE WE HAVE NO IMMEDIATE PLANS TO GO TO SUCROSE PERIOD&
Predicted: HWWVVJR C+MMA PAXPYYZZZZWWWWWWWXXXXORYYDFFJFRFNK CMMMM MORE CCCNOMIQ WASTTSTTSSW''TNNY'A'.DRXKKKKPPRIOD& 
Truth    : HOWEVER COMMA PEPSI IS ALWAYS EXPLORING DIFFERENT COMMA MORE ECONOMIC WAYS TO SWEETEN OUR DRINKS PERIOD&
Predicted: FF TH+ PRIZCSJFSSUICGGOWSTBBBCCMMS COMPEDITVVV WWTHHHYGHFFORKKOSP-QKGCGCUUU.CC..M.W.CCCH.SSD''FU.UUTUPTIONN PRPOO- DOUBL--QQOT 
Truth    : IF THE PRICE OF SUCROSE BECOMES COMPETITIVE WITH HIGH FRUCTOSE CORN SYRUP COMMA WE'D CONSIDER OUR OPTIONS PERIO

Epoch [11/11], Step [50/387], Per-char Loss: 0.5838
Time Elapsed:  0:01:14.148620
Epoch [11/11], Step [100/387], Per-char Loss: 0.5418
Time Elapsed:  0:02:27.716618


In [ ]:
testX = np.load('test_new.npy', encoding='bytes', allow_pickle=True)
testX = [torch.Tensor(k) for k in testX]
testX_lens = torch.LongTensor([k.shape[0] for k in testX])
testP = VocalDataset(testX, None, testX_lens, None)
testT = DataLoader(testP, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=8)


In [ ]:
def BeamSearch(SymbolSets, y_probs, BeamWidth):
    #SymbolSets: A list containing all the symbols (the vocabulary without blank)
    #y_probs: Numpy array with shape (# of symbols + 1, Seq_length, batch_size)
    #BeamWidth: Width of the beam.
    batch_size = y_probs.shape[2]

    def InitPaths():
        init_blankpath = [{} for i in range(batch_size)]
        init_path = [{} for i in range(batch_size)]

        for j in range(batch_size):
            y_slice = y_probs[:,0,j]
            for i in range(y_slice.shape[0]):
                if i == 0: init_blankpath[j][""] = y_slice[0]
                else: init_path[j][SymbolSets[i-1]] = y_slice[i]

        return init_blankpath, init_path

    def Prune(blank_paths, symbol_paths):
        paths = []
        for (k,v) in blank_paths.items():
            paths.append((k,v))

        for (k,v) in symbol_paths.items():
            paths.append((k,v))

        paths.sort(key=lambda l: l[1], reverse=True)

        new_blank_paths = {}
        new_symbol_paths = {}

        threshold = paths[BeamWidth-1][1] if len(paths) > BeamWidth else paths[-1][1]

        #print(paths, threshold)
        for (k,v) in blank_paths.items():
            if v >= threshold:
                new_blank_paths[k] = v

        for (k,v) in symbol_paths.items():
            if v >= threshold:
                new_symbol_paths[k] = v

        return new_blank_paths, new_symbol_paths

    def ExtendWithBlank(blank_paths, symbol_paths, y_slice):
        new_blank_paths = {}

        for (k,v) in blank_paths.items():
            # if k in new_blank_paths:
            #     new_blank_paths[k] += v * y_slice[0]
            # else:
            new_blank_paths[k] = v * y_slice[0]

        for (k,v) in symbol_paths.items():
            if k in new_blank_paths:
                new_blank_paths[k] += v * y_slice[0]
            else: new_blank_paths[k] = v * y_slice[0]

        return new_blank_paths

    def ExtendWithSymbol(blank_paths, symbol_paths, y_slice):
        new_symbol_paths = {}

        for (k,v) in blank_paths.items():
            for i in range(len(SymbolSets)):
                # if k+SymbolSets[i] in new_symbol_paths:
                #     new_symbol_paths[k+SymbolSets[i]] += v * y_slice[i+1]
                # else:
                new_symbol_paths[k+SymbolSets[i]] = v * y_slice[i+1]

        for (k,v) in symbol_paths.items():
            for i in range(len(SymbolSets)):
                nset = k+SymbolSets[i] if k[-1] != SymbolSets[i] else k
                if nset in new_symbol_paths:
                    new_symbol_paths[nset] += v * y_slice[i+1]
                else: new_symbol_paths[nset] = v * y_slice[i+1]

        return new_symbol_paths

    def MergeIdentPaths(blank_paths, symbol_paths):
        final_paths = {}

        for (k,v) in blank_paths.items():
            #tk = k.strip()
            tk = k
            if tk in final_paths:
                final_paths[tk] += v
            else:
                final_paths[tk] = v

        for (k,v) in symbol_paths.items():
            #tk = k.strip()
            tk = k
            if tk in final_paths:
                final_paths[tk] += v
            else:
                final_paths[tk] = v

        return final_paths

    blankpaths, paths = InitPaths()

    for i in range(1,y_probs.shape[1]):
        for j in range(batch_size):
            #Prune
            y_slice = y_probs[:,i,j]
            bp, sp = Prune(blankpaths[j], paths[j])
            #print(bp,sp,"\n")
            #Extend by a blank
            nbp = ExtendWithBlank(bp, sp, y_slice)
            #Extend by a symbol
            nsp = ExtendWithSymbol(bp, sp, y_slice)
            #print(nbp, nsp, "\n")
            blankpaths[j] = nbp
            paths[j] = nsp

    final_paths = []
    best_path = []

    for j in range(batch_size):
        #merge identical paths
        #pick best
        final_paths.append(MergeIdentPaths(blankpaths[j], paths[j]))
        best_path.append(sorted(final_paths[-1].items(),key=lambda l: l[1], reverse=True)[0])

    #print(sorted(final_paths[-1].items(),key=lambda l: l[1], reverse=True))
    return (best_path[0][0], final_paths[0])
    #return (best_path, final_paths)


In [ ]:
def output(model,file=None):
    submit = open("submission.csv","w")
    print("Id,Predicted",file=submit)
    if file is not None:
        model.load_state_dict(torch.load(file))

    model.eval()
    cd.empty_cache()

    idd = 0

    with torch.no_grad():
        for dta in tqdm(testT):
            voc, _, voc_len, _ = dta

            output = model(voc,voc_len, None, None)
            bpath, res = BeamSearch(charSetWithoutBlank,output.permute(2,1,0),10)

            for i in range(res.shape[0]):
                buf = bpath.replace("~"," ").replace("%","").replace("&","")
                buf = " ".join(buf.split(" "))
                buf = buf.replace(",","COMMA").replace("@","PERIOD").replace("{","DOUBLE-QUOTE").replace("[","SINGLE-QUOTE").replace(":","COLON").replace("-","HYPHEN").replace("(","LEFT-BRACE").replace(")","RIGHT_BRACE").replace("$","++MISC_SOUND++")
                print(str(idd)+","+buf, file=submit)
                idd += 1





In [ ]:
model.load_state_dict(torch.load("take2-1-05010436.pkl"))

<All keys matched successfully>

# Testing and auto submission

In [ ]:
output(model)
!/home/ubuntu/.local/bin/kaggle competitions submit -c 11-785-s20-hw3p2 -f submission.csv -m "test"

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



100%|██████████████████████████████████████| 42.0k/42.0k [00:01<00:00, 23.0kB/s]
Successfully submitted to 11-785, Spring 2020, Homework 3 Part 2